# Training a Linear Model (predict BMI off of close Height/Weight)

In [1]:
import torch
import copy
import syft as sy
from torch import nn
from torch import optim
hook = sy.TorchHook(torch)

In [2]:
alex = sy.VirtualWorker(hook, id='alex')
toby = sy.VirtualWorker(hook, id='toby')
jake = sy.VirtualWorker(hook, id='jake')
matt = sy.VirtualWorker(hook, id='matt')
shreyas = sy.VirtualWorker(hook, id='shreyas')

# secure worker node built as well for secure regression building
secure_machine = sy.VirtualWorker(hook, id='secure_machine')

In [3]:
# Preprocessing of making tensors, sending, making a grid

# initiate tensors; the data is weight & height
alex_data = torch.tensor([[165, 71]])
toby_data = torch.tensor([[175, 73]])
jake_data = torch.tensor([[123, 69]])
matt_data = torch.tensor([[137, 82]])
shreyas_data = torch.tensor([[167, 76]])

# Send the tensor to the right worker node
alex_tensor = alex_data.send(alex)
toby_tensor = toby_data.send(toby)
jake_tensor = jake_data.send(jake)
matt_tensor = matt_data.send(matt)
shreyas_tensor = shreyas_data.send(shreyas)


In [4]:
# this is BMI data
alex_outcome_data = torch.tensor([[23]])
toby_outcome_data = torch.tensor([[23]])
jake_outcome_data = torch.tensor([[18]])
matt_outcome_data = torch.tensor([[14]])
shreyas_outcome_data = torch.tensor([[21]])

alex_outcome = alex_outcome_data.send(alex)
toby_outcome = toby_outcome_data.send(toby)
jake_outcome = jake_outcome_data.send(jake)
matt_outcome = matt_outcome_data.send(matt)
shreyas_outcome = shreyas_outcome_data.send(shreyas)

In [5]:
model = nn.Linear(2, 1)
model

Linear(in_features=2, out_features=1, bias=True)

In [6]:

# modified from OpenMinded Pysyft Tutorial Part 04 - Federated Learning via a Trusted Aggregator
def train_model(iterations, worker_iterations):
    
    for iterator in range(iterations):
        
        toby_model = model.copy().send(toby)
        alex_model = model.copy().send(alex)
        jake_model = model.copy().send(jake)
        matt_model = model.copy().send(matt)
        shreyas_model = model.copy().send(shreyas)
        
        # create optimizer functions
        toby_opt = optim.SGD(params=toby_model.parameters(), lr=0.05)
        alex_opt = optim.SGD(params=alex_model.parameters(), lr=0.05)
        jake_opt = optim.SGD(params=jake_model.parameters(), lr=0.05)
        matt_opt = optim.SGD(params=matt_model.parameters(), lr=0.05)
        shreyas_opt = optim.SGD(params=shreyas_model.parameters(), lr=0.05)
        
        for wi in range(worker_iterations):
            
            # Train Toby's Model
            toby_opt.zero_grad()
            toby_pred = toby_model(toby_tensor.float())
            toby_loss = ((toby_pred - toby_outcome)**2).sum()
            toby_loss.backward()
            
            toby_opt.step()
            toby_loss = toby_loss.get().data
            
            # Train Alex's Model
            alex_opt.zero_grad()
            alex_pred = alex_model(alex_tensor.float())
            alex_loss = ((alex_pred - alex_outcome)**2).sum()
            alex_loss.backward()
            
            alex_opt.step()
            alex_loss = alex_loss.get().data
            
            # Train Jake's Model
            jake_opt.zero_grad()
            jake_pred = jake_model(jake_tensor.float())
            jake_loss = ((jake_pred - jake_outcome)**2).sum()
            jake_loss.backward()
            
            jake_opt.step()
            jake_loss = jake_loss.get().data
            
            # Train Matt's Model
            matt_opt.zero_grad()
            matt_pred = matt_model(matt_tensor.float())
            matt_loss = ((matt_pred - matt_outcome)**2).sum()
            matt_loss.backward()
            
            matt_opt.step()
            matt_loss = matt_loss.get().data
            
            # Train Shreyas's Model
            shreyas_opt.zero_grad()
            shreyas_pred = shreyas_model(shreyas_tensor.float())
            shreyas_loss = ((shreyas_pred - shreyas_outcome)**2).sum()
            shreyas_loss.backward()
            
            shreyas_opt.step()
            shreyas_loss = shreyas_loss.get().data
            
        # move the model to secure worker
        toby_model.move(secure_machine)
        jake_model.move(secure_machine)
        alex_model.move(secure_machine)
        matt_model.move(secure_machine)
        shreyas_model.move(secure_machine)
                        
        # set the weight and bias divided by number of models used
        with torch.no_grad():
            model.weight.set_(((toby_model.weight.data + jake_model.weight.data + alex_model.weight.data + matt_model.weight.data + shreyas_model.weight.data) / 5).get())
            model.bias.set_(((toby_model.bias.data + jake_model.bias.data + alex_model.bias.data + matt_model.bias.data + shreyas_model.bias.data) / 5).get())
            
        # printing loss
        print ("toby loss: " + str(toby_loss))
        print ("jake loss: " + str(jake_loss))
        print ("alex loss: " + str(alex_loss))
        print ("matt loss: " + str(matt_loss))
        print ("shreyas loss: " + str(shreyas_loss))
        
train_model(4, 1)



toby loss: tensor(398.4310)
jake loss: tensor(10.0474)
alex loss: tensor(269.2057)
matt loss: tensor(47.0936)
shreyas loss: tensor(280.0983)
toby loss: tensor(1.8444e+09)
jake loss: tensor(1.0145e+09)
alex loss: tensor(1.6563e+09)
matt loss: tensor(1.2927e+09)
shreyas loss: tensor(1.7287e+09)
toby loss: tensor(1.5862e+16)
jake loss: tensor(8.7592e+15)
alex loss: tensor(1.4249e+16)
matt loss: tensor(1.1176e+16)
shreyas loss: tensor(1.4885e+16)
toby loss: tensor(1.3666e+23)
jake loss: tensor(7.5466e+22)
alex loss: tensor(1.2276e+23)
matt loss: tensor(9.6290e+22)
shreyas loss: tensor(1.2824e+23)


# Testing model on workers

In [7]:
# this can see how well the model worked
predictions = model(alex_data.float())
loss = ((predictions - alex_outcome_data) ** 2).sum()
print(predictions)
print(loss.data)

# this can see how well the model worked
predictions = model(toby_data.float())
loss = ((predictions - toby_outcome_data) ** 2).sum()
print(predictions)
print(loss.data)

tensor([[1.0284e+15]], grad_fn=<AddmmBackward>)
tensor(1.0577e+30)
tensor([[1.0851e+15]], grad_fn=<AddmmBackward>)
tensor(1.1774e+30)
